In [1]:
#%matplotlib notebook

In [2]:

import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import warnings; warnings.simplefilter('ignore')
import gmaps
import json
import requests
import pandas as pd
import numpy as np
import inspect
#from citipy import citipy
#import matplotlib.pyplot as plt
from scipy import stats
from config import api_key
import pandas as pd
import seaborn as sns

import scipy.stats as st

ModuleNotFoundError: No module named 'config'

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
abnb_data = pd.read_csv("Resources/airbnb_data.csv")

In [ ]:
Cities = []
for city in abnb_df['City']:
    if city not in Cities:
        Cities.append(city)

In [ ]:
cities = [city.lower() for city in Cities]

In [ ]:
states = ['NC', 'TX', 'MA', 'FL', 'MA', 'IL', 'NV', 'OH', 'CO', 'NJ', 'CA', 'TN', 'LA', 'NY', 'CA', 'CA', 'OR', 'OR', 'CA', 'CA', 'CA', 'CA', 'CA', 'WA', 'DC']

In [ ]:
loop_df = pd.DataFrame({
    'city': cities,
    'City': Cities,
    'State': states
})

loop_df['Population'] = ''

In [ ]:
for index, row in loop_df.iterrows():
    city = row["city"]
    City = row["City"]
    state = row["State"]
    url = f'http://public.opendatasoft.com/api/records/1.0/search/?dataset=cities-and-towns-of-the-united-states%40public&q={city}&facet=name&facet=state&refine.name={City}&refine.state={state}'
    response = requests.get(url).json()
    print(f"Searching population record for {city}")
    
    try:
        loop_df.loc[index, "Population"] = response['records'][0]['fields']['pop_2010']
    
        print(f"{city}'s population: {response['records'][0]['fields']['pop_2010']}")
    
        print('________________________________________________')
    
    except (KeyError, IndexError):
        print(f"Population for {city} not found... Continue scanning.")
        
        print('________________________________________________')

In [ ]:
graphs_draft_df = abnb_df.groupby('City').mean().reset_index()
graphs_draft_df['Population'] = loop_df['Population']
graphs_draft_df['State'] = loop_df['State']
graphs_df = graphs_draft_df[['City', 'State', 'Population', 'Latitude', 'Longitude', 'Price', 'Minimum Nights', 'Number of Reviews', 'Reviews per Month', 'Availability (days per year)']]
graphs_df['Population'] = pd.to_numeric(graphs_df['Population'], errors='coerce')

## Create bar graph to show population by city

In [ ]:
sorted_by_pop = graphs_df.sort_values(by='Population', ascending=False)

pop_city = sorted_by_pop[['City', 'Population']]
pop_city = pop_city.set_index('City')

pop_city.plot(kind="bar", figsize=(10,7))

# Set a title for the chart
plt.title('Population by City')
plt.xlabel("Cities")
plt.ylabel("Population (in millions)")
plt.show()
plt.tight_layout()

In [ ]:
# Set x axis and tick locations
x_axis = np.arange(len(sorted_by_pop))
tick_locations = [value for value in x_axis]

# Create a list indicating where to write x labels and set figure size to adjust for space
plt.figure(figsize=(25,3))
plt.bar(x_axis, sorted_by_pop['Population'], color='r', alpha=0.5, align="center")
plt.xticks(tick_locations, sorted_by_pop['City'], rotation="vertical")

# Set x and y limits
plt.xlim(-0.75, len(x_axis))
plt.ylim(0, max(sorted_by_pop['Population'])+10)

# Set a Title and labels
plt.title('Population by City')
plt.xlabel("Cities")
plt.ylabel("Population (in millions)")

# Save our graph and show the grap
plt.tight_layout()
plt.savefig('Resources/pop_by_city.png')
plt.show()

In [ ]:
# Create Fig and gridspec
fig = plt.figure(figsize=(16, 10), dpi= 80)
grid = plt.GridSpec(4, 4, hspace=0.5, wspace=0.2)

# Define the axes
ax_main = fig.add_subplot(grid[:-1, :-1])
ax_right = fig.add_subplot(grid[:-1, -1], xticklabels=[], yticklabels=[])
ax_bottom = fig.add_subplot(grid[-1, 0:-1], xticklabels=[], yticklabels=[])

# Scatterplot on main ax
graphs_df('Reviews per Month', 'Price',  c=graphs_df.State.astype('City').cat.codes, alpha=.9, data=graphs_df, cmap="Set1", edgecolors='black', linewidths=.5)

# Add a graph in each part
sns.boxplot(graphs_df.hwy, ax=ax_right, orient="v")
sns.boxplot(graphs_df.displ, ax=ax_bottom, orient="h")

# Decorations ------------------
# Remove x axis name for the boxplot
ax_bottom.set(xlabel='')
ax_right.set(ylabel='')

# Main Title, Xlabel and YLabel
ax_main.set(title='Scatterplot with Histograms \n displ vs hwy', xlabel='displ', ylabel='hwy')

# Set font size of different components
ax_main.title.set_fontsize(20)
for item in ([ax_main.xaxis.label, ax_main.yaxis.label] + ax_main.get_xticklabels() + ax_main.get_yticklabels()):
    item.set_fontsize(14)

plt.show()